In [2]:
from pyxtal import Group, Wyckoff_position
import torch

In [3]:
wp: Wyckoff_position = Group(225).Wyckoff_positions[0]
wp.ops

[Rot:
 [[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0.  1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[-1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 1.  0.  0.]
  [ 0. -1.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[0. 0. 1.]
  [1. 0. 0.]
  [0. 1. 0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  0.  1.]
  [-1.  0.  0.]
  [ 0. -1.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  0. -1.]
  [-1.  0.  0.]
  [ 0.  1.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  0. -1.]
  [ 1.  0.  0.]
  [ 0. -1.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0. -1.  0.]
  [ 0.  0.  1.]
  [-1.  0.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  1.  0.]
  [ 0.  0. -1.]
  [-1.  0.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0. -1.  0.]
  [ 0.  0. -1.]
  [ 1.  0.  0.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0.  1.  0.]
  [ 1.  0.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:
 [[ 0. -1.  0.]
  [-1.  0.  0.]
  [ 0.  0. -1.]]
 tau
 [0. 0. 0.],
 Rot:

In [38]:
from dataclasses import dataclass
from matplotlib import axes
import numpy as np
from pymatgen.core.operations import SymmOp
import itertools
from typing import Self

@dataclass(eq=False)
class Op:
    """A symmetry operation with added functionality."""
    def as_symmop(self) -> SymmOp:
        raise NotImplementedError()

    def __call__(self, pt):        
        return self.as_symmop().operate(pt)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:
        raise NotImplementedError()
    
    def __repr__(self):
        return '{}[{}]'.format(
            self.__class__.__name__.title(),
            self.as_symmop().as_xyz_str()
        )
    
    def order(self) -> int:
        """Smallest n such that self ^ n = the identity operation."""
        op = self.as_symmop()
        # simplest way to test equality is a random asymmetric point
        # apply the op first: this lets projections work
        pt = op.operate([np.pi / 4, np.sqrt(3) / 3, np.tanh(0.2)]) % 1
        image = pt.copy()
        for i in range(1, 25):
            image = op.operate(image) % 1
            if np.allclose(image, pt):
                return i
        
        raise ValueError('Could not find order: {}', self)
    

    def __eq__(self, other):
        return hash(self) == hash(other)
    
    def inverse(self):
        raise NotImplementedError()
    
    def __mul__(self, other):        
        return Composition([self, other])
    
    def __div__(self, other):
        return Composition([self, other.inverse()])
    
    def __pow__(self, expon):
        if expon == 0:
            return Identity()
        elif expon < 0:
            return self.inverse() ** (-expon)
        else:
            return Composition([self] * expon)
    
    def __hash__(self):
        return hash(','.join('{:.4f}'.format(x % 1) for x in self([np.pi / 4, np.sqrt(3) / 3, np.tanh(0.2)])))
    

@dataclass(repr=False, eq=False)
class Composition(Op):
    """A composition of ops."""
    ops: list[Op]

    def as_symmop(self) -> SymmOp:
        start = self.ops[0].as_symmop()
        for next in self.ops[1:]:
            start = start * next.as_symmop()

        return start
    
    def __mul__(self, other) -> Self:
        if hasattr(other, 'ops'):
            return Composition(self.ops + other.ops)
        else:
            return Composition(self.ops + [other])
        
    def __rmul__(self, other) -> Self:
        if hasattr(other, 'ops'):
            return Composition(other.ops + self.ops)
        else:
            return Composition([other] + self.ops)
        
    def inverse(self):
        return Composition([op.inverse() for op in self.ops[::-1]])

@dataclass(repr=False, eq=False)
class Identity(Op):
    """The general position x,y,z."""
    def as_symmop(self) -> SymmOp:
        return SymmOp.from_xyz_str('x,y,z')
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:
        return [cls()]
    
    def inverse(self):
        return self

@dataclass(repr=False, eq=False)
class Permutation(Op):
    """A permutation of axes, like y, x, z or 0, z, y."""
    axes: list[int]

    def as_symmop(self) -> SymmOp:
        mat = np.eye(3)
        for ax1, ax2 in enumerate(self.axes):            
            if ax1 != ax2:
                mat[ax2, ax1] = 1            
                mat[ax1, ax1] = 0            

        return SymmOp.from_rotation_and_translation(mat)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:
        free_axes = compute_free_axes(wp)
        if len(free_axes) <= 1:
            return []
        elif len(free_axes) == 2:
            ax1, ax2 = free_axes
            perm = [0, 1, 2]
            perm[ax1], perm[ax2] = perm[ax2], perm[ax1]
            perm = cls(perm)
            if has_op(wp, perm):
                return [perm]
            else:
                return []
        else:
            # there are 6 potential permutations:
            # [0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]
            # identity doesn't matter
            # [2, 0, 1] and [1, 2, 0] are redundant with each other: each is the other squared
            # the rest are swaps of two axes
            # unfortunately, 24e of no. 212, as an example, has (z, x, y)
            # and (y, z, x) but not any others: have to have all of these.
            all_perms = [cls(axes) for axes in [
                [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 1, 0]
            ]]
            return [p for p in all_perms if has_op(wp, p)]
        
    def inverse(self):
        return Permutation([self.axes.index(a) for a in range(3)])

    
def ops_equal(op1: SymmOp, op2: SymmOp) -> bool:
    pt = np.array([np.pi / 4, np.sqrt(3) / 3, np.tanh(0.2)])
    return np.allclose(op1.operate(pt) % 1, op2.operate(pt) % 1)

def compute_free_axes(wp: Wyckoff_position) -> list[int]:
    return [ax for ax in range(3) if ax not in wp.get_frozen_axis()]

def has_op(wp: Wyckoff_position, op: Op) -> bool:
    symm_op = op.as_symmop()
    return any(ops_equal(symm_op, wp_op) for wp_op in wp.ops)

p = Permutation.find_all(Group(212).Wyckoff_positions[0])[0]
p

Permutation[z, x, y]

In [39]:
@dataclass(repr=False, eq=False)
class Translation(Op):
    """A translation of a fraction of a unit cell."""        
    tau: list[float]
    def as_symmop(self) -> SymmOp:
        return SymmOp.from_rotation_and_translation(translation_vec=self.tau)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:
        free_axes = compute_free_axes(wp)
        translations = []
        for op in wp.ops:
            if np.allclose(np.diag(op.rotation_matrix)[free_axes], 1):
                tau = op.translation_vector
                if not np.allclose(tau, 0):
                    trans = cls(tau)
                    if trans not in translations:
                        translations.append(cls(tau))
        return translations
    
    def inverse(self):
        return Translation(-self.tau)
    
Translation.find_all(wp)

[Translation[x, y+1/2, z+1/2],
 Translation[x+1/2, y, z+1/2],
 Translation[x+1/2, y+1/2, z]]

In [40]:
@dataclass(repr=False, eq=False)
class Reflection(Op):
    """A reflection around one or many axes."""
    axes: list[float]
    def as_symmop(self) -> SymmOp:
        mat = np.eye(3)
        for ax in self.axes:
            mat[ax, ax] = -1
        return SymmOp.from_rotation_and_translation(mat)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:
        free_axes = compute_free_axes(wp)
        reflect_axes = []
        for ax in free_axes:
            if has_op(wp, cls([ax])):
                reflect_axes.append([ax])
        
        for axs in itertools.chain(itertools.combinations(free_axes, 2), itertools.combinations(free_axes, 3)):
            if not all([ax] in reflect_axes for ax in axs) and has_op(wp, cls(axs)):
                reflect_axes.append(axs)

        return [cls(axs) for axs in reflect_axes]
    
    def inverse(self):
        return self
    
Reflection.find_all(wp)

[Reflection[-x, -y, z], Reflection[-x, y, -z], Reflection[x, -y, -z]]

In [41]:
from pymatgen.core.operations import SymmOp


@dataclass(repr=False, eq=False)
class Rotation(Op):
    """A catch-all for rotations that don't fit into other classes."""
    # only ever need integers here
    matrix: list[list[int]]

    def as_symmop(self) -> SymmOp:
        return SymmOp.from_rotation_and_translation(self.matrix)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:        
        rotations = []
        for op in wp.ops:
            if not np.allclose(op.translation_vector, 0):
                continue

            # the rules:
            # can't be permutation (all 1s)
            # can be permutation + reflection, but only if components aren't in the list
            # can't be identity
            # can be anything more complicated like x-y,y,-z
            ii, jj = op.rotation_matrix.nonzero()
            vals = op.rotation_matrix[(ii, jj)]
            if np.allclose(vals, 1):
                continue  # permutation/identity
            elif np.allclose(ii, jj):
                continue # simple reflection
            else:
                rotations.append(cls(op.rotation_matrix))
        return rotations
    
    def inverse(self):
        return Rotation(np.linalg.inv(self.matrix).round(2))
    
Rotation.find_all(Wyckoff_position.from_group_and_letter(181, 'k'))

[Rotation[x-y, -y, -z], Rotation[-x+y, y, -z]]

In [46]:
from pymatgen.core.operations import SymmOp


@dataclass(repr=False, eq=False)
class Glide(Op):
    """A glide plane (reflection + translation)"""
    reflection: Reflection
    translation: Translation

    def as_symmop(self) -> SymmOp:
        return SymmOp.from_rotation_and_translation(self.reflection.as_symmop().rotation_matrix, self.translation.tau)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:        
        free_axes = compute_free_axes(wp)
        glides = []
        for op in wp.ops:
            if np.allclose(op.translation_vector, 0):
                continue
            if np.allclose(op.rotation_matrix, np.diag(np.diag(op.rotation_matrix))):
                # rotation matrix is diagonal
                diag = np.diag(op.rotation_matrix)[free_axes]
                if not np.allclose(diag, 1) and np.allclose(abs(diag), 1):
                    # at least one reflection
                    # now check if sub-operations aren't present
                    # print(op.rotation_matrix, op.translation_vector)
                    ref = Reflection([x for x in free_axes if diag[x] == -1])
                    trans = Translation(op.translation_vector)
                    # print(ref, trans)
                    if not has_op(wp, ref) or not has_op(wp, trans):
                        glides.append(cls(ref, trans))

        return glides
    
    def inverse(self):   
        # TODO replace with real calculation     
        return self.translation.inverse() * self.reflection.inverse()
    
gs = Glide.find_all(Wyckoff_position.from_group_and_letter(92, 'b'))
gs

[Glide[-x, -y, z+1/2],
 Glide[-x+1/2, y+1/2, -z+1/4],
 Glide[x+1/2, -y+1/2, -z+3/4]]

In [47]:
from pymatgen.core.operations import SymmOp


@dataclass(repr=False, eq=False)
class Screw(Op):
    """A rotation plus translation, without either being valid alone."""
    rotation: Rotation
    translation: Translation

    def as_symmop(self) -> SymmOp:
        return SymmOp.from_rotation_and_translation(self.rotation.matrix, self.translation.tau)
    
    @classmethod
    def find_all(cls, wp: Wyckoff_position) -> list[Self]:        
        screws = []
        glides = Glide.find_all(wp)
        for op in wp.ops:
            if np.allclose(op.translation_vector, 0):
                continue

            # copied from the rotation code
            # the rules:
            # can't be permutation (all 1s)
            # can be permutation + reflection, but only if components aren't in the list
            # can't be identity
            # can be anything more complicated like x-y,y,-z
            ii, jj = op.rotation_matrix.nonzero()
            vals = op.rotation_matrix[(ii, jj)]
            if np.allclose(vals, 1):
                continue  # permutation/identity
            elif np.allclose(ii, jj):
                continue # simple reflection
            else:
                rot = Rotation(op.rotation_matrix)
                tau = Translation(op.translation_vector)
                if not has_op(wp, rot) or not has_op(wp, tau):
                    screws.append(cls(rot, tau))

        
        return [s for s in screws if s not in glides]
    
    def inverse(self):
        # TODO replace with real calculation
        return self.translation.inverse() * self.rotation.inverse()
        
            
screws = Screw.find_all(Wyckoff_position.from_group_and_letter(92, 'b'))
print([s.order() for s in screws])
screws

[4, 4, 2]


[Screw[-y+1/2, x+1/2, z+1/4],
 Screw[y+1/2, -x+1/2, z+3/4],
 Screw[-y, -x, -z+1/2]]

In [69]:
from functools import reduce

def op_prod(ops: list[Op]) -> Op:
    return reduce(lambda x, y: x * y, ops)

def free_group(gens: list[Op]) -> set[Op]:
    all_gens = {Identity()}
    for gen in gens:
        curr = gen
        for _ in range(gen.order()):
            all_gens.add(curr)
            curr = curr * gen

    ops = all_gens.copy()
    new_ops = {op_prod(prod) for prod in itertools.product(all_gens, repeat=2)}
    while ops != new_ops:
        ops = new_ops
        new_ops = {op_prod(prod) for prod in itertools.product(ops, repeat=2)}
    return ops

def compute_generating_set(wp: Wyckoff_position):
    all_gens = []
    # the order here matters!
    # it determines which generators get priority
    # screws/glides can generate translations, but not vice versa
    # so translations go last
    for op_type in [Permutation, Reflection, Rotation, Glide, Screw, Translation]:
        all_gens.extend(sorted(op_type.find_all(wp), key=lambda x: -x.order()))
    
    ops = {Identity()}
    gens = [Identity()]
    for gen in all_gens:                
        if gen not in ops:
            gens.append(gen)
            ops = free_group(gens)
    
    return gens[1:], ops, all_gens

wp = Wyckoff_position.from_group_and_letter(210, 'h')
# print(wp)
gens, ops, all_gens = compute_generating_set(wp)
print(len(ops))
print(gens)
print(all_gens)

96
[Permutation[z, x, y], Reflection[-x, -y, z], Screw[y+1/4, -x+3/4, z+3/4]]
[Permutation[z, x, y], Reflection[-x, -y, z], Reflection[-x, y, -z], Reflection[x, -y, -z], Screw[y+1/4, -x+3/4, z+3/4], Screw[-y+3/4, x+3/4, z+1/4], Screw[x+3/4, z+1/4, -y+3/4], Screw[x+1/4, -z+3/4, y+3/4], Screw[z+3/4, y+1/4, -x+3/4], Screw[-z+3/4, y+3/4, x+1/4], Screw[y+3/4, x+3/4, -z+5/4], Screw[-y+1/4, -x+3/4, -z+3/4], Screw[y+1/4, -x+5/4, z+5/4], Screw[-y+3/4, x+5/4, z+3/4], Screw[x+3/4, z+3/4, -y+5/4], Screw[x+1/4, -z+5/4, y+5/4], Screw[z+3/4, y+3/4, -x+5/4], Screw[z+1/4, -y+5/4, x+5/4], Screw[-z+3/4, y+5/4, x+3/4], Screw[-z+1/4, -y+3/4, -x+3/4], Screw[y+5/4, x+1/4, -z+5/4], Screw[-y+3/4, -x+1/4, -z+3/4], Screw[y+3/4, -x+3/4, z+5/4], Screw[-y+5/4, x+3/4, z+3/4], Screw[x+5/4, z+1/4, -y+5/4], Screw[-x+5/4, z+3/4, y+3/4], Screw[-x+3/4, -z+1/4, -y+3/4], Screw[x+3/4, -z+3/4, y+5/4], Screw[z+5/4, y+1/4, -x+5/4], Screw[-z+5/4, y+3/4, x+3/4], Screw[y+3/4, -x+5/4, z+3/4], Screw[-y+5/4, x+5/4, z+1/4], Screw[x+5/

In [70]:
xx = np.linspace(0, 1, 20)
yy = np.linspace(0, 1, 20)
zz = np.linspace(0, 1, 20)

fixed_points = [[] for _ in range(len(gens))]
for point in itertools.product(xx, yy, zz):
    for fixed_set, gen in zip(fixed_points, gens):
        if np.allclose(gen(point), point):
            fixed_set.append(point)

[len(fs) for fs in fixed_points]

[20, 20, 0]

In [115]:
from sympy import *

x, y, z = symbols('x y z', real=True)

op = gens[2].as_symmop()

transformed = op.operate([x, y, z])

conditions = []    
for ax in range(3):
    conditions.append((transformed[ax] - [x, y, z][ax]) % 1)

sols = solve(conditions, [x, y, z])
# if sols:
#     for i, var in enumerate((x, y, z)):
#         if var not in sols and conditions[i] != 0:
#             sols[var] = []
#         else:
#             sols[var] = var
#     print(sols)

In [116]:
sols

[]

In [93]:
lone_star = np.array([[0.0593, 0.0812, 0.1342, 1]]).T
# def fixed_matrix(op: Op | SymmOp) -> np.array:
#     """Finds the affine matrix such that A @ (x, y, z, 1) = 0 implies op((x, y, z)) = op((x, y, z))."""
#     if isinstance(op, Op):
#         op = op.as_symmop()
op = gens[1].as_symmop()
    
M = op.affine_matrix
# for a, b, c in (-1, 0, 1):
# M @ (pt, 1) = (a, b, c) + (pt, 1)
# (M - np.diag(a, b, c)) @ (pt, 1) = 0
constrs = []
values = []
for a, b, c in itertools.product((-1, 0, 1), repeat=3):
    M_abc = M - np.diag([a, b, c, 1])
    constrs.append(M_abc)
    values.append(M_abc @ lone_star)

sorted_i = np.argsort(values)
constrs = np.array(constrs)[sorted_i]
values = np.array(values)[sorted_i]    
for i in range(len(values) - 1):
    if values[i] <= 0 and values[i + 1] > 0:
        break
        # return constrs[i]
print(constrs[i])


fixed_matrix(gens[-1]) @ np.array([0.5, 0.25, 0, 1]).T

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [95]:
constrs

array([[[[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]]],



       [[[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]]],


        [[[-3.,  0.,  0.,  0.],
          [ 0., -3.,  0.,  0.],
          

In [90]:
    for a, b, c in itertools.product((-1, 0, 1), repeat=3):
        M_abc = M - np.eye(4) + np.diag([a, b, c, 0])
        constrs.append(M_abc)
        values.append(M_abc @ lone_star)

array([[0.0593],
       [0.0812],
       [0.1342],
       [1.    ]])

In [ ]:
import pickle

# from tqdm import trange
# all_wps = []
# for g in trange(2, 231):
#     for wp in Group(g).Wyckoff_positions:
#         if not any(wp.has_equivalent_ops(wp2) for wp2 in all_wps):
#             all_wps.append(wp)

# print(len(all_wps))
# with open('all_wps.pkl', 'wb') as out:
#     pickle.dump(all_wps, out)

with open('all_wps.pkl', 'rb') as infile:
    all_wps: list[Wyckoff_position] = pickle.load(infile)

In [ ]:
from tqdm import tqdm

# demonstrates that no ops are unable to be generated

# for wp in tqdm(all_wps):
#     if wp.get_dof() == 3:
#         gens, ops, all_gens = compute_generating_set(wp)
#         if len(ops) < len(wp.ops):
#             if len(free_group(gens, 5)) < len(wp.ops):
#                 print(wp)
#                 print(len(ops))
#                 print(gens)
#                 raise ValueError('Whoops!')                
#         elif len(ops) > len(wp.ops):
#             print(wp)
#             bad_ops = [op for op in ops if not any(ops_equal(op.as_symmop(), wp_op) for wp_op in wp.ops)]
#             print(bad_ops)
#             raise ValueError(f'Too many ops: {len(ops)} > {len(wp.ops)}')

In [ ]:
for wp in tqdm(all_wps):
    if wp.get_dof() == 2:
        gens, ops, all_gens = compute_generating_set(wp)
        if len(ops) < len(wp.ops):
            if len(free_group(gens, 5)) < len(wp.ops):
                print(wp)
                print(len(ops))
                print(gens)
                raise ValueError('Whoops!')                
        elif len(ops) > len(wp.ops):
            print(wp)
            bad_ops = [op for op in ops if not any(ops_equal(op.as_symmop(), wp_op) for wp_op in wp.ops)]
            print(bad_ops)
            raise ValueError(f'Too many ops: {len(ops)} > {len(wp.ops)}')

  0%|          | 0/939 [00:00<?, ?it/s]/tmp/ipykernel_2750484/1466670839.py:78: DeprecationWarning: `from_xyz_string` is deprecated!
Use from_xyz_str instead
  return SymmOp.from_xyz_string('x,y,z')
  2%|▏         | 19/939 [00:00<00:00, 7636.24it/s]

Wyckoff position 1b in space group 6 with site symmetry .m.
x, 1/2, z
[Translation[x, y+1/2, z], Identity[x, y, z]]


ValueError: Too many ops: 2 > 1

In [ ]:
# # only rotational symmetries are simple reflections

# for i, wp in enumerate(all_wps):
#     for op in wp.ops:
#         if not np.allclose(op.rotation_matrix, np.round(op.rotation_matrix)):
#             print(op)

In [ ]:
# # show that the maximum denominator is 12

# from math import gcd

# denoms = []
# wycks = []
# for i, wp in enumerate(all_wps):
#     for trans in Translation.find_all(wp):
#         for c in trans.tau:
#             c60 = 240 * c
#             if abs(c60 - round(c60)) >= 0.01:
#                 raise ValueError('Bad fraction!', c)
#             else:
#                 denoms.append(240 // gcd(int(round(c60)), 240))
#                 wycks.append(i)

# wycks = np.array(wycks)
# denoms = np.array(denoms)
# assert set(denoms) == {1, 2, 3, 4, 6, 8, 12}
# wycks[denoms == 12]